In [2]:
## VISSIM Modules
import win32com.client as com
import os
## FA Modules
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("ERROR: GPU DEVICE NOT FOUND.")

# Import Keras is TF 1.    
#from keras.models import load_model

import sys
sys.path.append('../')

## Data Management Modules
import pickle
from collections import defaultdict
## Math Modules
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import math


ERROR: GPU DEVICE NOT FOUND.


In [4]:
## Our Modules
from COMServer import COMServerDispatch
import Agents 
import Simulation_Environments as SE

ModuleNotFoundError: No module named 'Agents'

In [5]:
# reload modules for debugging
from imp import reload
reload(Agents)
reload(SE)
#reload(AC)

NameError: name 'Agents' is not defined

In [6]:
class Agent():     
    # Initialize agent with dimension of state and action space
    def __init__(self,state_size, action_size,actions):
        # Number of states, action space and memory
        self.state_size = state_size
        self.action_size = action_size
        self.actions = actions

    # Choose and action
    def Action(self, state, actions=None):
        if actions is None:
            actions = self.actions
        passactions[act_idx]
    
    # Learning routine
    def Learn(self,sarsa):
        pass
    
    # learn from a batch
    def Learn_Batch(self,Sarsa, batch_size=32):
        pass

In [7]:
# Q_learner
'''
Easy Q_learner Q_Function
'''
class Q_function(Agent):
    def __init__(self, actions = None):
        # Q function
        self.Q = defaultdict(lambda: defaultdict(float))
        # number of visits
        self.N = defaultdict(lambda: defaultdict(float))
        self.actions = actions

    def Check(self,state,actions=None):
        if actions is None :
            actions = self.actions
        
        if state not in self.Q.keys():
            for action in actions:
                self.Q[state][action] = 0

    def Max(self,state):
        Q_maximum = np.max(list(self.Q[state].values()))
        return Q_maximum

    def Action(self,state,epsilon=0):
        if np.random.rand() < epsilon :
            idx = np.random.randint(len(actions))
            action = actions[idx]
        else :
            self.Check(state,actions)
            action = max(self.Q[state], key=self.Q[state].get)
        return action

    def Learn(self,sars,learning_rate=0.1,discount_factor=0.5):
        state, action, reward, next_state = sars
        # Check if state,action and next_state are in Q
        self.Check(state)
        self.Check(next_state)
        self.N_update(state,action)

        dQ = reward \
            + discount_factor * self.Max(next_state) \
            - self.Q[state][action]
        self.Q[state][action] = self.Q[state][action] + learning_rate * dQ 
        
        return self.Q

    def N_update(self,state,action,actions=None):
        if actions is None :
            actions = self.actions
        
        if state not in self.N.keys():
            for action in actions:
                self.N[state][action] = 0 
        self.N[state][action] = self.N[state][action] + 1
        return self.N[state][action]

    def Print(self):
        for state in Q_fn.Q.keys():
            for action in Q_fn.Q[state].keys():
                print(state,action,Q_fn.N[state][action],Q_fn.Q[state][action])

## Setup Environment

In [8]:
model_name  = 'Single_Cross_Straight'
vissim_working_directory = 'C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\Vissim\\'
#vissim_working_directory = 'C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\0_TMF\\MLforFlowOptimisation\\Vissim\\'
actions = [(1,0,1,0),\
            (0,1,0,1)]

In [9]:
Env= SE.Q_Sim_Env(None,model_name,vissim_working_directory)

NameError: name 'SE' is not defined

In [ ]:
# Some code to reload Simulator instance (without rebooting Vissim)
# Vissim = Env.Vissim
#Env = SE.Q_Sim_Env(Vissim,model_name,vissim_working_directory)


In [8]:
# MANUAL LOAD (part 1)
import win32com.client as com
import os

# Dispatch works better than dynamoc dispatch
#Vissim = com.Dispatch("Vissim.Vissim")

for _ in range(5):
    try:
        Vissim = com.Dispatch("Vissim.Vissim")
        print('success!')
        break
    except:
        print('fail')
    

fail
fail
fail
fail
success!


In [17]:
# MANUAL LOAD (part 2)
input_file = 'C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\Vissim\\Single_Cross_Straight\\Single_Cross_Straight.inpx'
Vissim.LoadNet(input_file)
Env = SE.Q_Sim_Env(Vissim,model_name,vissim_working_directory)

## MaxWeight

In [18]:
MW = Agents.MaxWeight(4,2,actions)

In [20]:
action = actions[0]
for _ in range(25):
    state, reward, done = Env.Step(action)
    action = MW.Action(state[0])

## Q-Learning

In [ ]:
Q_fn = Q_function(actions)

In [ ]:
action = actions[0]
state, reward, done = Env.Step(action)

for _ in range(10):
    next_state, reward, done = Env.Step(action) 
    Q_fn.Learn([tuple(state[0]),action,reward, tuple(next_state[0])])
    state = next_state
    action = Q_fn.Action(tuple(state[0]),1.0)

#### Debug

## Raymond Code

In [ ]:
# Raymond Agent
from collections import deque


#Code adapted from http://inoryy.com/post/tensorflow2-deep-reinforcement-learning/

import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as kl
import tensorflow.keras.losses as kls
import tensorflow.keras.optimizers as ko


class ProbabilityDistribution(tf.keras.Model):
    def call(self, logits):
        # sample a random categorical action from given logits
        return tf.squeeze(tf.random.categorical(logits, 1), axis=-1)

class Model(tf.keras.Model):
    def __init__(self, num_actions):
        super().__init__('mlAC.model.action_value(state)p_policy')
        # no tf.get_variable(), just simple Keras API
        self.hidden1 = kl.Dense(128, activation='relu')
        self.hidden2 = kl.Dense(128, activation='relu')
        self.value = kl.Dense(1, name='value')
        # logits are unnormalized log probabilities
        self.logits = kl.Dense(num_actions, name='policy_logits')
        self.dist = ProbabilityDistribution()

    def call(self, inputs):
        # inputs is a numpy array, convert to Tensor
        x = tf.convert_to_tensor(inputs, dtype=tf.float32)
        # separate hidden layers from the same input tensor
        hidden_logs = self.hidden1(x)
        hidden_vals = self.hidden2(x)
        return self.logits(hidden_logs), self.value(hidden_vals)

    def action_value(self, obs):
        # executes call() under the hood
        logits, value = self.predict(obs)
        action = self.dist.predict(logits)
        # a simpler option, will become clear later why we don't use it
        # action = tf.random.categorical(logits, 1)
        return action , value
  
    
class ACAgent(Agent):

    def __init__(self,\
                 state_size,\
                 action_size,\
                 # ID,\
                 # state_type,\
                 # npa,\
                 n_step_size=32,\
                 gamma=0.99,\
                 alpha=0.000065): #,
                 #Vissim):
                


        print("Deploying instance of Actor_Critic Agent(s)")

        #just temporary
        self.epsilon=0

        # Model
        # hyperparameters for loss terms and Agent
        self.params = {'value': 0.5, 'entropy': 0.0001, 'gamma': gamma}
        self.model = Model(action_size)
        self.model.compile(
            optimizer=ko.RMSprop(lr=alpha),
            # define separate losses for policy logits and value estimate
            loss=[self._logits_loss, self._value_loss]
        )

#         # Agent Junction ID and Controller ID
#         self.signal_id = ID
#         self.signal_controller = npa.signal_controllers[self.signal_id]
#         self.signal_groups = npa.signal_groups[self.signal_id]

        # Number of states, action space and memory
        self.state_size = state_size
        self.action_size = action_size


        # Potential actions (compatible phases) and transitions
        self.update_counter = 1                                 # Timesteps until next update
        self.compatible_actions = [[0,1,0,1],[1,0,1,0]]         # Potential actions (compatible phases), 1 means green

        # Internal State Traffic Control Variables
        self.intermediate_phase = False                         # Boolean indicating an ongoing green-red or red-green transition
        self.transition_vector = []                             # Vector that will store the transitions between updates


        # Initial Setup of S, A, R, S_
        self.state = np.reshape([0,0,0,0], [1,state_size])
        self.newstate = np.reshape([0,0,0,0], [1,state_size])
        self.action = 0
        self.newaction = 0
        self.reward = 0

        # Metrics Storage Initialization
        self.episode_reward = []
        self.loss = []

        # The memory will store (state , action , reward, next_state) in a batch
        self.memory = deque(maxlen=n_step_size)
        self.n_step_size = n_step_size

    # Add memory on the right, if over memory limit, pop leftmost item
    def remember(self, state, action, reward, next_state):
        self.memory.append([state, action, reward, next_state])

#     # Update the Junction IDs for the agent
#     def update_IDS(self, ID, npa):
#         self.signal_id = ID
#         self.signal_controller = npa.signal_controllers[self.signal_id]
#         self.signal_groups = npa.signal_groups[self.signal_id]

    # Need to test before loading to build the graph (surely an other way to do it ...)
    def test(self):
        _,_ = self.model.action_value(np.empty((1,self.state_size)))


    def _value_loss(self, returns, value):
        # value loss is typically MSE between value estimates and returns
        return self.params['value']*kls.mean_squared_error(returns, value)

    def _logits_loss(self, acts_and_advs, logits):
        # a trick to input actions and advantages through same API
        actions, advantages = tf.split(acts_and_advs, 2, axis=-1)
        # sparse categorical CE loss obj that supports sample_weight arg on call()
        # from_logits argument ensures transformation into normalized probabilities
        weighted_sparse_ce = kls.SparseCategoricalCrossentropy(from_logits=True)
        # policy loss is defined by policy gradients, weighted by advantages
        # note: we only calculate the loss on the actions we've actually taken
        actions = tf.cast(actions, tf.int32)
        policy_loss = weighted_sparse_ce(actions, logits, sample_weight=advantages)
        # entropy loss can be calculated via CE over itself
        entropy_loss = kls.categorical_crossentropy(logits, logits, from_logits=True)
        # here signs are flipped because optimizer minimizes
        return policy_loss - self.params['entropy']*entropy_loss

    def _returns_advantages(self, rewards, values, next_value):
        # next_value is the bootstrap value estimate of a future state (the critic)
        returns = np.append(np.zeros_like(rewards), next_value, axis=-1)
        # returns are calculated as discounted sum of future rewards
        for t in reversed(range(rewards.shape[0])):
            returns[t] = rewards[t] + self.params['gamma'] * returns[t+1]
        returns = returns[:-1]
        # advantages are returns - baseline, value estimates in our case
        advantages = returns - values
        return returns, advantages

    def choose_action(self, state):
        action, _ = self.model.action_value(state)
        return np.squeeze(action, axis=-1)

    #Performing step of gradient descent on the agent memory
    def learn(self):


        Sample = np.array(self.memory)

        states, actions, rewards, next_state  = np.concatenate(Sample[:,0], axis=0), Sample[:,1].astype('int32'), Sample[:,2], np.concatenate(Sample[:,3], axis=0)



        _, values = self.model.action_value(states)
        values = values.squeeze()

        _, next_value  = self.model.action_value(next_state)

        next_value = next_value[-1]



        returns, advs = self._returns_advantages(rewards, values, next_value)

        # a trick to input actions and advantages through same API

        acts_and_advs = np.concatenate([actions[:, None], advs[:, None]], axis=-1)

        # performs a full training step on the collected batch
        # note: no need to mess around with gradients, Keras API handles it
        losses = self.model.train_on_batch(states, [acts_and_advs, returns])

        #print(losses)


## comments
    # def train(self, env, batch_sz=32, updates=1000):
    #     # storage helpers for a single batch of data
    #     actions = np.empty((batch_sz,), dtype=np.int32)
    #     rewards, dones, values = np.empty((3, batch_sz))
    #     observations = np.empty((batch_sz,) + env.observation_space.shape)
    #     # training loop: collect samples, send to optimizer, repeat updates times
    #     ep_rews = [0.0]
    #     next_obs = env.reset()
    #     for update in range(updates):
    #         for step in range(batch_sz):
    #             observations[step] = next_obs.copy()
    #             actions[step], values[step] = self.model.action_value(next_obs[None, :])
    #             next_obs, rewards[step], dones[step], _ = env.step(actions[step])

    #               ep_rews[-1] += rewards[step]
    #             if dones[step]:
    #                 ep_rews.append(0.0)
    #                 next_obs = env.reset()

    #           _, next_value = self.model.action_value(next_obs[None, :])
    #         returns, advs = self._returns_advantages(rewards, dones, values, next_value)
    #         # a trick to input actions and advantages through same API
    #         acts_and_advs = np.concatenate([actions[:, None], advs[:, None]], axis=-1)
    #         # performs a full training step on the collected batch
    #         # note: no need to mess around with gradients, Keras API handles it
    #         losses = self.model.train_on_batch(observations, [acts_and_advs, returns])
    #     return ep_rews

In [ ]:
AC = ACAgent(4,2)

In [ ]:
AC.model.action_value(np.empty((1,AC.state_size)))

In [ ]:
act_idx = 0
state, reward, done = Env.Step(actions[act_idx])

In [ ]:
for _ in range(100):
    next_state, reward, done = Env.Step(actions[act_idx]) 
    AC.remember(state,act_idx,reward, next_state)
    state = next_state
    act_idx = AC.model.action_value(state)[0][0]
    AC.learn()

## Raymond 2nd Agent

In [ ]:
# Raymond Agent II
from collections import deque


#Code adapted from http://inoryy.com/post/tensorflow2-deep-reinforcement-learning/

import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as kl
import tensorflow.keras.losses as kls
import tensorflow.keras.optimizers as ko


from collections import deque


#Code adapted from http://inoryy.com/post/tensorflow2-deep-reinforcement-learning/

import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as kl
import tensorflow.keras.losses as kls
import tensorflow.keras.optimizers as ko


class ProbabilityDistribution(tf.keras.Model):
    def call(self, logits):
        # sample a random categorical action from given logits
        return tf.squeeze(tf.random.categorical(logits, 1), axis=-1)

class Model(tf.keras.Model):
    def __init__(self, num_actions):
        super().__init__('mlp_policy')
        # no tf.get_variable(), just simple Keras API

        self.core1 = kl.Dense(32, activation='relu')
        
        self.value1 = kl.Dense(42, activation='relu', name='value1') #64
        self.value2 = kl.Dense(42, activation='relu', name='value2')
        self.value3 = kl.Dense(1, name='value3')
        # logits are unnormalized log probabilities


        self.logits1 = kl.Dense(42, activation='relu', name='policy_logits1')
        self.logits2 = kl.Dense(42, activation='relu', name='policy_logits2')
        self.logits3 = kl.Dense(num_actions, name='policy_logits3')

        self.dist = ProbabilityDistribution()

    def call(self, inputs):
        # inputs is a numpy array, convert to Tensor
        x = tf.convert_to_tensor(inputs, dtype=tf.float32)

        # This it the core of the model
        x = self.core1(x)
        
        # separate hidden layers from the core
        hidden_logs = self.logits1(x)
        hidden_logs = self.logits2(hidden_logs)

        hidden_vals = self.value1(x)
        hidden_vals = self.value2(hidden_vals)

        return self.logits3(hidden_logs), self.value3(hidden_vals)

    def action_value(self, obs):
        # executes call() under the hood
        logits, value = self.predict(obs)
        action = self.dist.predict(logits)
        # a simpler option, will become clear later why we don't use it
        # action = tf.random.categorical(logits, 1)
        return action , value


# An working model training with entropy = 0.00001 en nstep = 32 and learn every step lr = 0.000065 gama = 0.99 
class Modelsave1(tf.keras.Model):
    def __init__(self, num_actions):
        super().__init__('mlp_policy')

        # no tf.get_variable(), just simple Keras API
        self.hidden1 = kl.Dense(42, activation='relu')
        self.hidden2 = kl.Dense(42, activation='relu')
        self.value = kl.Dense(1, name='value')
        # logits are unnormalized log probabilities
        self.logits = kl.Dense(num_actions, name='policy_logits')
        self.dist = ProbabilityDistribution()

    def call(self, inputs):
        # inputs is a numpy array, convert to Tensor
        x = tf.convert_to_tensor(inputs, dtype=tf.float32)

        # This it the core of the model
       
        # separate hidden layers from the core
        hidden_logs = self.hidden1(x)
        hidden_vals = self.hidden2(x)
        return self.logits(hidden_logs), self.value(hidden_vals)

    def action_value(self, obs):
        # executes call() under the hood
        logits, value = self.predict(obs)
        action = self.dist.predict(logits)
        # a simpler option, will become clear later why we don't use it
        # action = tf.random.categorical(logits, 1)
        return action , value
  
    
class ACAgent_II:

    def __init__(self,\
                 state_size,\
                 action_size,\
                 #ID,\
                 #state_type,\
                 #npa,\
                 n_step_size =32,\
                 gamma = 0.99,\
                 alpha = 0.000065,\
                 entropy = 0.00001 ):
                    #Vissim):


        print("Deploying instance of Actor_Critic Agent(s) !!! TENSORFLOW 2 IS NEEDED !!! ")
        # agent type flag 
        self.type = 'AC'



        #just temporary
        self.epsilon = 0

        self.trainstep = 0

        # Model
        # hyperparameters for loss terms and Agent
        self.params = {'value': 0.5, 'entropy': entropy, 'gamma': gamma}
        self.model = Modelsave1(action_size)
        self.model.compile(
            optimizer=ko.RMSprop(lr=alpha),
            # define separate losses for policy logits and value estimate
            loss=[self._logits_loss, self._value_loss]
        )

#         # Agent Junction ID and Controller ID
#         self.signal_id = ID
#         self.signal_controller = npa.signal_controllers[self.signal_id]
#         self.signal_groups = npa.signal_groups[self.signal_id]

        # Number of states, action space and memory
        self.state_size = state_size
        self.action_size = action_size

        # Potential actions (compatible phases) and transitions
        self.update_counter = 1                                 # Timesteps until next update
        if self.action_size == 2:
            self.compatible_actions = [[0,1,0,1],[1,0,1,0]]         # Potential actions (compatible phases), 1 means green
        elif self.action_size == 8:
            self.compatible_actions = [[1,1,1,0,0,0,0,0,0,0,0,0],
                                        [0,0,0,1,1,1,0,0,0,0,0,0],
                                        [0,0,0,0,0,0,1,1,1,0,0,0],
                                        [0,0,0,0,0,0,0,0,0,1,1,1],
                                        [1,0,0,0,0,0,1,0,0,0,0,0],
                                        [0,0,0,1,0,0,0,0,0,1,0,0],
                                        [0,1,1,0,0,0,0,1,1,0,0,0],
                                        [0,0,0,0,1,1,0,0,0,0,1,1]]
        else:
            raise Exception("ERROR: Wrong Action Size. Please review master settings and RLAgents.py")
        # Internal State Traffic Control Variables
        self.intermediate_phase = False                         # Boolean indicating an ongoing green-red or red-green transition
        self.transition_vector = []                             # Vector that will store the transitions between updates


        # Initial Setup of S, A, R, S_
        self.state = np.zeros((1,state_size))
        self.newstate = np.zeros((1,state_size))
        self.action = 0
        self.newaction = 0
        self.reward = 0

        # Metrics Storage Initialization
        self.episode_reward = []
        self.loss = []
        self.queues_over_time = [[0,0,0,0]]
        self.accumulated_delay= [0]


        # The memory will store (state , action , reward, next_state) in a batch
        self.memory = deque(maxlen=n_step_size)
        self.n_step_size = n_step_size

    # Add memory on the right, if over memory limit, pop leftmost item
    def remember(self, state, action, reward, next_state):
        self.memory.append([state, action, reward, next_state])

    # Update the Junction IDs for the agent
    def update_IDS(self, ID, npa):
        self.signal_id = ID
        self.signal_controller = npa.signal_controllers[self.signal_id]
        self.signal_groups = npa.signal_groups[self.signal_id]

    # Need to test before loading to build the graph (surely an other way to do it ...)
    def test(self):
        _,_ = self.model.action_value(np.empty((1,self.state_size)))


    def _value_loss(self, returns, value):
        # value loss is typically MSE between value estimates and returns
        return self.params['value']*kls.mean_squared_error(returns, value)

    def _logits_loss(self, acts_and_advs, logits):
        # a trick to input actions and advantages through same API
        actions, advantages = tf.split(acts_and_advs, 2, axis=-1)
        # sparse categorical CE loss obj that supports sample_weight arg on call()
        # from_logits argument ensures transformation into normalized probabilities
        weighted_sparse_ce = kls.SparseCategoricalCrossentropy(from_logits=True)
        # policy loss is defined by policy gradients, weighted by advantages
        # note: we only calculate the loss on the actions we've actually taken
        actions = tf.cast(actions, tf.int32)
        policy_loss = weighted_sparse_ce(actions, logits, sample_weight=advantages)
        # entropy loss can be calculated via CE over itself
        entropy_loss = kls.categorical_crossentropy(logits, logits, from_logits=True)
        # here signs are flipped because optimizer minimizes
        return policy_loss - self.params['entropy']*entropy_loss

    def _returns_advantages(self, rewards, values, next_value):
        # next_value is the bootstrap value estimate of a future state (the critic)
        returns = np.append(np.zeros_like(rewards), next_value, axis=-1)
        # returns are calculated as discounted sum of future rewards
        for t in reversed(range(rewards.shape[0])):
            returns[t] = rewards[t] + self.params['gamma'] * returns[t+1]
        returns = returns[:-1]
        # advantages are returns - baseline, value estimates in our case
        advantages = returns - values
        return returns, advantages

    def choose_action(self, state):
        action, _ = self.model.action_value(state)
        return np.squeeze(action, axis=-1)

    #Performing step of gradient descent on the agent memory
    def learn(self):


        Sample = np.array(self.memory)

        states, actions, rewards, next_state  = np.concatenate(Sample[:,0], axis=0), Sample[:,1].astype('int32'), Sample[:,2], np.concatenate(Sample[:,3], axis=0)



        _, values = self.model.action_value(states)
        values = values.squeeze()

        _, next_value  = self.model.action_value(next_state)

        next_value = next_value[-1]



        returns, advs = self._returns_advantages(rewards, values, next_value)

        # a trick to input actions and advantages through same API

        acts_and_advs = np.concatenate([actions[:, np.newaxis], advs[:, np.newaxis]], axis=-1)

        # performs a full training step on the collected batch
        # note: no need to mess around with gradients, Keras API handles it
        losses = self.model.train_on_batch(states, [acts_and_advs, returns])

        #print(losses)



    # def train(self, env, batch_sz=32, updates=1000):
    #     # storage helpers for a single batch of data
    #     actions = np.empty((batch_sz,), dtype=np.int32)
    #     rewards, dones, values = np.empty((3, batch_sz))
    #     observations = np.empty((batch_sz,) + env.observation_space.shape)
    #     # training loop: collect samples, send to optimizer, repeat updates times
    #     ep_rews = [0.0]
    #     next_obs = env.reset()
    #     for update in range(updates):
    #         for step in range(batch_sz):
    #             observations[step] = next_obs.copy()
    #             actions[step], values[step] = self.model.action_value(next_obs[None, :])
    #             next_obs, rewards[step], dones[step], _ = env.step(actions[step])

    #               ep_rews[-1] += rewards[step]
    #             if dones[step]:
    #                 ep_rews.append(0.0)
    #                 next_obs = env.reset()

    #           _, next_value = self.model.action_value(next_obs[None, :])
    #         returns, advs = self._returns_advantages(rewards, dones, values, next_value)
    #         # a trick to input actions and advantages through same API
    #         acts_and_advs = np.concatenate([actions[:, None], advs[:, None]], axis=-1)
    #         # performs a full training step on the collected batch
    #         # note: no need to mess around with gradients, Keras API handles it
    #         losses = self.model.train_on_batch(observations, [acts_and_advs, returns])
    #     return ep_rews

In [ ]:
AC = ACAgent_II(4,2)

In [ ]:
AC.model.action_value(np.empty((1,AC.state_size)))

In [ ]:
act_idx = 0
state, reward, done = Env.Step(actions[act_idx])

In [ ]:
for _ in range(100):
    next_state, reward, done = Env.Step(actions[act_idx]) 
    AC.remember(state,act_idx,reward, next_state)
    state = next_state
    act_idx = AC.model.action_value(state)[0][0]
    AC.learn()

#### Raymond Debug

## Debug

## Alvaro DQN

Main changes:

    -- changed Action function name
        
    -- get state and get reward have been passed to the simulator
    
    
    -- various default parameters are set 
    
  
    -- removed state, npa, type as that is a simulator property

In [ ]:
# Alvaro DQN agent
'''
Vissim dependencies are:
    ID --    required for 
                __init__
                update_IDS(self, ID, npa)
    
    npa --   required for same as ID
    
    Vissim -- is not even used

RECOMENDATION: 
    Keep agents in a dictionary, with key as a generic ID
    Run update_IDS as a simulation function

'''

'''
self.compatible_actions

    -- actions specific to network should not be defined internally. 
    
RECOMENDATION:
    compatible_actions -- variable defined at __init__
'''

'''
UNUSED VARIABLES

self.accumulated_delay
self.intermediate_phase = False                        
self.transition_vector = [] 
Vissim
state_type

RECOMENDATION:
    remove
'''

'''
VARIABLES NEVER UPDATED

self.episode_reward = []

RECOMMENDATION:
    remove
'''

'''
REDUNDANT VARIABLES

self.state 
self.newstate 
self.action 
self.newaction 
self.reward 

RECOMMENDATION
    can keep but is not clear what that are doing
'''

from collections import deque
import numpy as np
import random
import PER

import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.keras.models import load_model, Sequential, Model
from tensorflow.keras.layers import Dense, Input, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.losses as kls
from tensorflow.keras import regularizers

class DQNAgent:
    def __init__(self,\
                 state_size,\
                 action_size,\
                 ID,\
                 state_type,\
                 npa,\
                 memory_size,\
                 gamma,\
                 epsilon,\
                 alpha,\
                 copy_weights_frequency,\
                 Vissim,\
                 PER_activated,\
                 DoubleDQN,\
                 Dueling):
        
        # Agent Junction ID and Controller ID
        self.signal_id = ID
        self.signal_controller = npa.signal_controllers[self.signal_id]
        self.signal_groups = npa.signal_groups[self.signal_id]

        # Number of states, action space and memory
        self.state_size = state_size
        self.action_size = action_size

        # Agent Hyperparameters
        self.gamma = gamma                  # discount rate
        self.epsilon = epsilon              # exploration rate
        self.learning_rate = alpha          # learning rate

        # Agent Architecture
        self.DoubleDQN = DoubleDQN            # Double Deep Q Network Flag
        self.Dueling = Dueling                # Dueling Q Networks Flag
        self.PER_activated = PER_activated    # Prioritized Experience Replay Flag
        self.type = 'DQN'                     # Type of the agent

        # Model and target networks
        self.copy_weights_frequency = copy_weights_frequency    # Frequency to copy weights to target network
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())

        # Potential actions (compatible phases) and transitions
        self.update_counter = 1                                 # Timesteps until next update
        if self.action_size == 2:
            self.compatible_actions = [[0,1,0,1],[1,0,1,0]]         # Potential actions (compatible phases), 1 means green
        elif self.action_size == 4:
        	self.compatible_actions = [[1,1,1,0,0,0,0,0,0,0,0,0],
                                        [0,0,0,1,1,1,0,0,0,0,0,0],
                                        [0,0,0,0,0,0,1,1,1,0,0,0],
                                        [0,0,0,0,0,0,0,0,0,1,1,1]]
        elif self.action_size == 8:
            self.compatible_actions = [[1,1,1,0,0,0,0,0,0,0,0,0],
                                        [0,0,0,1,1,1,0,0,0,0,0,0],
                                        [0,0,0,0,0,0,1,1,1,0,0,0],
                                        [0,0,0,0,0,0,0,0,0,1,1,1],
                                        [1,0,0,0,0,0,1,0,0,0,0,0],
                                        [0,0,0,1,0,0,0,0,0,1,0,0],
                                        [0,1,1,0,0,0,0,1,1,0,0,0],
                                        [0,0,0,0,1,1,0,0,0,0,1,1]]
        else:
            raise Exception("ERROR: Wrong Action Size. Please review master settings and RLAgents.py")

        # Internal State Traffic Control Variables
        self.intermediate_phase = False                         # Boolean indicating an ongoing green-red or red-green transition
        self.transition_vector = []                             # Vector that will store the transitions between updates

        # Architecture Debug Messages
        if self.DoubleDQN:
            if self.Dueling:
                print("Deploying instance of Dueling Double Deep Q Learning Agent(s)")
            else:
                print("Deploying instance of Double Deep Q Learning Agent(s)")
        else:
            if self.Dueling:
                print("Deploying instance of Dueling Deep Q Learning Agent(s)")
            else:
                print("Deploying instance of Standard Deep Q Learning Agent(s)")

        # Initial Setup of S, A, R, S_
        self.state = np.zeros((1,state_size))
        self.newstate = np.zeros((1,state_size))
        self.action = 0
        self.newaction = 0
        self.reward = 0
        
        # Metrics Storage Initialization
        self.episode_reward = []
        self.loss = []
        self.queues_over_time = [[0,0,0,0]]
        self.accumulated_delay= [0]
        
        if self.PER_activated:
            # If PER_activated spawn BinaryTree and Memory object to store priorities and experiences
            self.memory = PER.Memory(memory_size)
        else:
            # Else use the deque structure to only store experiences which will be sampled uniformly
            self.memory = deque(maxlen=memory_size)

    # Update the Junction IDs for the agent
    def update_IDS(self, ID, npa):
        self.signal_id = ID
        self.signal_controller = npa.signal_controllers[self.signal_id]
        self.signal_groups = npa.signal_groups[self.signal_id]
    
    # Agent Neural Network definition
    def _build_model(self):
        if self.Dueling:
            # Architecture for the Neural Net in the Dueling Deep Q-Learning Model
            #model = Sequential()
            input_layer = Input(shape = (self.state_size,))
            dense1 = Dense(64, input_dim=self.state_size, activation='relu')(input_layer)
            #dense2 = Dense(48, activation='relu')(dense1)
            #flatten = Flatten()(dense2)
            fc1 = Dense(48)(dense1)
            dueling_actions = Dense(self.action_size)(fc1)
            fc2 = Dense(48)(dense1)
            dueling_values = Dense(1)(fc2)

            def dueling_operator(duel_input):
                duel_v = duel_input[0]
                duel_a = duel_input[1]
                return (duel_v + (duel_a - K.mean(duel_a, axis = 1, keepdims = True)))

            policy = Lambda(dueling_operator, name = 'policy')([dueling_values, dueling_actions])
            model = Model(inputs=[input_layer], outputs=[policy])
            model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
            return(model)
        else:
            # Architecture for the Neural Net in Deep-Q learning Model (also Double version)
            model = Sequential()
            model.add(Dense(42, input_dim=self.state_size, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
            model.add(Dense(42, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
            model.add(Dense(42, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
            model.add(Dense(self.action_size, activation='linear',kernel_regularizer=regularizers.l2(0.01)))
            
            #model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
            model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate, epsilon =1.5*10**-4))
            return model
    
    # Add memory on the right, if over memory limit, pop leftmost item
    def remember(self, state, action, reward, next_state):
        if self.PER_activated:
            experience = np.array([state, action, reward, next_state])
            self.memory.store(experience)
        else:
            self.memory.append([state, action, reward, next_state])
    
    # Choosing actions
    def choose_action(self, state):
        if np.random.rand() <= self.epsilon:
            action = random.randrange(self.action_size)
            #print('Chosen Random Action {}'.format(action+1))
        else:
            act_values = self.model.predict(state)
            action = np.argmax(act_values[0])
            #print('Chosen Not-Random Action {}'.format(action+1))
        return action
    
    
    
    # Sample a batch of "batch_size" experiences and perform 1 step of gradient descent on all of them simultaneously
    def learn_batch(self, batch_size, episode):
        state_vector = []
        target_f_vector = []
        absolute_errors = [] 

        if self.PER_activated:
            tree_idx, minibatch, ISWeights_mb = self.memory.sample(batch_size)
        else:
            idx = np.random.randint(len(self.memory), size=batch_size,dtype=int)
            minibatch = np.array(self.memory)[idx]
        
        
        state, action, reward, next_state = np.concatenate(minibatch[:,0], axis=0 ), minibatch[:,1].astype('int32') ,minibatch[:,2].reshape(batch_size,1), np.concatenate( minibatch[:,3] , axis=0 )
        
        
        
        if self.DoubleDQN:
            next_action = np.argmax(self.model.predict(np.reshape(next_state,(batch_size,self.state_size))), axis=1)
            target = reward + self.gamma * self.target_model.predict(np.reshape(next_state,(batch_size,self.state_size)))[np.arange(batch_size),next_action].reshape(batch_size,1)
        else:
            # Fixed Q-Target
            target = reward + self.gamma * np.max(self.target_model.predict(np.reshape(next_state,(batch_size,self.state_size))),axis=1).reshape(batch_size,1)
            print(target.shape)
            # No fixed targets version
            # target = reward + self.gamma * np.max(self.model.predict(np.reshape(next_state,(1,self.state_size))))    
        
            
        # There should be a way to vectorize this
        #for state, action, reward, next_state in minibatch:
        #    if self.DoubleDQN:
        #        next_action = np.argmax(self.model.predict(np.reshape(next_state,(1,self.state_size))), axis=1)
        #        target = reward + self.gamma * self.target_model.predict(np.reshape(next_state,(1,self.state_size)))[0][next_action][0]
        #    else:
                # Fixed Q-Target
        #        target = reward + self.gamma * np.max(self.target_model.predict(np.reshape(next_state,(1,self.state_size))))
                # No fixed targets version
                # target = reward + self.gamma * np.max(self.model.predict(np.reshape(next_state,(1,self.state_size))))

        # This section incorporates the reward into the prediction and calculates the absolute error between old and new
        target_f = self.model.predict(state)
        
        absolute_errors = np.abs(target_f[np.arange(batch_size),action].reshape(batch_size,1)-target)
        
        #absolute_errors.append(abs(target_f[0][action] - target))
        
        target_f[np.arange(batch_size),action] = target.reshape(batch_size)
        
        
        #self.model.fit(state_matrix, target_f_matrix, epochs=1, verbose=0)
        self.model.fit(state, target_f, epochs=1, verbose=2,batch_size=batch_size)
        
        self.loss.append(self.model.history.history['loss'])

        if self.PER_activated:
            #Update priority
            self.memory.batch_update(tree_idx, absolute_errors)

        # Copy weights every "copy_weights_frequency" episodes
        #if (episode+1) % self.copy_weights_frequency == 0 and episode != 0:
        #    self.copy_weights()   

    # Copy weights function
    def copy_weights(self):
        self.target_model.set_weights(self.model.get_weights())
        print("Weights succesfully copied to Target model.")  

In [ ]:
'''
COMMENTS ON ALVARO MAIN FILE

action_type 
    -- where is this used
    
agents_deployed
    -- is this needed?
'''

In [ ]:
# parameters
PER_activated = True
batch_size = 64
memory_size = 1024
alpha   = 0.000065
gamma   = 0.95

## State-Action Parameters
state_size = 4
action_size = 8

agent_type = 'DuelingDDQN'        # DQN, DuelingDQN, DDQN, DuelingDDQN
reward_type = 'Queues'   
# 'Queues'          Sum of the queues for all lanes in intersection
# 'QueuesDiff'      Difference in queue lengths in last timestep
# 'QueuesDiffSC'    10000* QueuesDiff - Queues^2
# 'TotalDelayDiff'

state_type  = 'Queues'    # 'Queues', 'Delays', 'QueuesDifference' 'QueuesSpeedavrOccuperateSig' 'QueuesSig'
Random_Seed = 42

In [ ]:
from NParser import NetworkParser
npa = NetworkParser(Vissim)
######################################################################################################################
## Nested data structure:
## 
## Signal Controllers = signal_controllers[signal_controller_ids]
## Signal Groups      = signal_groups     [signal_controller_ids] [signal_group_id]
## Signal Heads       = signal_heads      [signal_controller_ids] [signal_heads_id]
## Lanes              = lanes             [signal_controller_ids] [signal_heads_id] [lane_id]
##
######################################################################################################################

In [ ]:
# Exploration Schedule ("linear" or "geometric")
episodes = 4

exploration_schedule = "geometric"
epsilon_start = 1
epsilon_end   = 0.001
if exploration_schedule == "linear":
    epsilon_decay = 1.2*(epsilon_end - epsilon_start)/(episodes-1)
    epsilon_sequence = [1 + epsilon_decay * entry for entry in range(episodes+1)]
    epsilon_sequence = [0 if entry < 0 else entry for entry in epsilon_sequence]
elif exploration_schedule == "geometric":
    epsilon_decay = np.power(epsilon_end/epsilon_start, 1./(episodes-1)) # Geometric decay
    epsilon_sequence = [epsilon_start * epsilon_decay ** entry for entry in range(episodes+1)]
else:
    print("ERROR: Unrecognized choice of exploration schedule.")

In [ ]:
copy_weights_frequency = 2

Agents = [DQNAgent(state_size,\
                   action_size,\
                   ID,\
                   state_type,\
                   npa,\
                   memory_size,\
                   gamma,\
                   epsilon_sequence[0],\
                   alpha,\
                   copy_weights_frequency,\
                   Vissim, PER_activated,\
                   DoubleDQN = True if agent_type == "DDQN" or agent_type =="DuelingDDQN" else False,\
                   Dueling = False if agent_type == "DQN" or agent_type == "DDQN" else True)\
          for ID in npa.signal_controllers_ids] 


In [ ]:
'''
demand_list 
    -- why do I need the demand list to populate the memory? 
'''
demand_change_timesteps = 450
demand = {"h":600, 'm':300, 'l':150}
demand_list = [[demand['l'], demand['l']], [demand['m'], demand['l']],\
              [demand['h'], demand['l']], [demand['h'], demand['m']],\
              [demand['h'], demand['h']], [demand['m'], demand['h']],
              [demand['l'], demand['h']], [demand['l'], demand['m']]]

In [ ]:
'''

seconds_per_green 
    -- do we need to control with 
    -- why is it used twice in populate_memory
'''

import Simulator_Functions as SF
timesteps_per_second = 1
Session_ID = 'BLAH'
seconds_per_green = 6


SF.Set_Quickmode(Vissim, timesteps_per_second)
memory, Agents, runflag = SF.prepopulate_memory(Agents, Vissim, state_type, reward_type, state_size, memory_size,\
                                                        vissim_working_directory, model_name, Session_ID,\
                                                        seconds_per_green, seconds_per_green, timesteps_per_second,\
                                                        demand_list, demand_change_timesteps, PER_activated)

In [ ]:
'''
log_prgess 
    -- why do I need this?
    
COMServerReload
    -- why do I need this?
    
mode 
    -- why do a need a mode flag in the internals here
'''

from COMServer import COMServerReload
from Utilities import log_progress, pltlive

simulation_length = 3600*1 + 1
mode = 'training'
seconds_per_yellow = 3.

reward_storage = []
best_agent_weights = []
best_agent_memory = []
reward_plot = np.zeros([episodes,])
loss_plot = np.zeros([episodes,])

SaveResultsAgent = False

# Iterations of the simulation
for episode in log_progress(range(episodes), every=1):

    # Reload map if it has already been run (previous episode or prepopulation)
    if episode !=0 or runflag == True:
        Simulation, Network = COMServerReload(Vissim, model_name, vissim_working_directory,\
                                              simulation_length, timesteps_per_second, delete_results = True)

        # Run Network Parser and ensure agents are linked to their intersections
        npa = NetworkParser(Vissim)
        for index, agent in enumerate(Agents):
            agent.update_IDS(agent.signal_id, npa)
            agent.episode_reward = []

    # Change the random seed
    Random_Seed += 1
    Vissim.Simulation.SetAttValue('RandSeed', Random_Seed)

    # Run Episode at maximum speed
    SF.Select_Vissim_Mode(Vissim, mode)
    SF.run_simulation_episode(Agents, Vissim, state_type, reward_type, state_size, simulation_length, timesteps_per_second,\
                              seconds_per_green, seconds_per_yellow, demand_list, demand_change_timesteps, mode,\
                              PER_activated)

    # Calculate episode average reward
    reward_storage, average_reward = SF.average_reward(reward_storage, Agents, episode, episodes)
    best_agent_weights, best_agent_memory = SF.best_agent(reward_storage, average_reward,\
                                                          best_agent_weights, best_agent_memory,\
                                                          vissim_working_directory, model_name, Agents, Session_ID)


    # Train agent with experience of episode and copy weights when necessary
    # Update exploration rate
    for agent in Agents: 
        for _ in range(5):
            agent.learn_batch(batch_size, episode)
    # Copy weights 
        if (episode+1) % agent.copy_weights_frequency == 0 and episode != 0:
            agent.copy_weights()
        agent.epsilon = epsilon_sequence[episode+1]

    # Security save for long trainings
    if SaveResultsAgent:
        if (episode+1)%partial_save_at == 0:
            SF.save_agents(vissim_working_directory, model_name, Agents, Session_ID, reward_storage)
            print('Saved Partial results at the end of episode {}.'.format(episode+1))

#Saving agents memory, weights and optimizer
if SaveResultsAgent:
    SF.save_agents(vissim_working_directory, model_name, Agents, Session_ID, reward_storage)
    print("Model, architecture, weights, optimizer, memory and training results succesfully saved.\
    Succesfully Terminated.")



#### Debug

In [ ]:
SF.green_red_to_amber(Agents[0], 6)

In [ ]:
Vissim.Simulation.SimulationStep()

# DEBUG

# DQN LITE

In [ ]:
        # Potential actions (compatible phases) and transitions
        self.update_counter = 1                                 # Timesteps until next update
        if self.action_size == 2:
            self.compatible_actions = [[0,1,0,1],[1,0,1,0]]         # Potential actions (compatible phases), 1 means green
        elif self.action_size == 4:
        	self.compatible_actions = [[1,1,1,0,0,0,0,0,0,0,0,0],
                                        [0,0,0,1,1,1,0,0,0,0,0,0],
                                        [0,0,0,0,0,0,1,1,1,0,0,0],
                                        [0,0,0,0,0,0,0,0,0,1,1,1]]
        elif self.action_size == 8:
            self.compatible_actions = [[1,1,1,0,0,0,0,0,0,0,0,0],
                                        [0,0,0,1,1,1,0,0,0,0,0,0],
                                        [0,0,0,0,0,0,1,1,1,0,0,0],
                                        [0,0,0,0,0,0,0,0,0,1,1,1],
                                        [1,0,0,0,0,0,1,0,0,0,0,0],
                                        [0,0,0,1,0,0,0,0,0,1,0,0],
                                        [0,1,1,0,0,0,0,1,1,0,0,0],
                                        [0,0,0,0,1,1,0,0,0,0,1,1]]
        else:
            raise Exception("ERROR: Wrong Action Size. Please review master settings and RLAgents.py")


In [ ]:
# DQN-LITE

'''
This is a light-weight version of Alvaro's DQN agent
'''

'''
MAIN CHANGES:
    -- is independent of network parser
    -- no junction ID is used 
    -- no dependence on VISSIM
    -- actions list is not maintained.
    -- separate agents for DDQN, PER etc...
'''

PER_activated = True
batch_size = 64
memory_size = 1024
alpha   = 0.000065
gamma   = 0.95

from collections import deque
import numpy as np
import random
import PER

import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.keras.models import load_model, Sequential, Model
from tensorflow.keras.layers import Dense, Input, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.losses as kls
from tensorflow.keras import regularizers

class DQNAgent:
    def __init__(self,\
                 epsilon,\
                 alpha = 0.000065,\
                 state_size=4,\
                 action_size=2,\
                 memory_size = 1024,\
                 gamma = 0.95,\
                 copy_weights_frequency =2
                ):
        

        # Number of states, action space and memory
        self.state_size = state_size
        self.action_size = action_size

        # Agent Hyperparameters
        self.gamma = gamma                  # discount rate
        self.epsilon = epsilon              # exploration rate
        self.learning_rate = alpha          # learning rate

        # Agent Architecture
        self.DoubleDQN = DoubleDQN            # Double Deep Q Network Flag
        self.Dueling = Dueling                # Dueling Q Networks Flag
        self.PER_activated = PER_activated    # Prioritized Experience Replay Flag
        self.type = 'DQN'                     # Type of the agent

        # Model and target networks
        self.copy_weights_frequency = copy_weights_frequency    # Frequency to copy weights to target network
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())

        self.memory = deque(maxlen=memory_size)
    
    # Agent Neural Network definition
    def _build_model(self):
            # Architecture for the Neural Net in Deep-Q learning Mode
            model = Sequential()
            model.add(Dense(42, input_dim=self.state_size, activation='tanh',kernel_regularizer=regularizers.l2(0.01)))
            model.add(Dense(42, activation='tanh',kernel_regularizer=regularizers.l2(0.01)))
            model.add(Dense(42, activation='tanh',kernel_regularizer=regularizers.l2(0.01)))
            model.add(Dense(self.action_size, activation='linear',kernel_regularizer=regularizers.l2(0.01)))
            
            model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
            return model
    
    def remember(self, state, action, reward, next_state):
            self.memory.append([state, action, reward, next_state])
    
    # Choosing actions
    def choose_action(self, state):
        if np.random.rand() <= self.epsilon:
            action = random.randrange(self.action_size)
        else:
            act_values = self.model.predict(state)
            action = np.argmax(act_values[0])
        return action
    
    
    
    # Sample a batch of "batch_size" experiences and perform 1 step of gradient descent on all of them simultaneously
    def learn_batch(self, episode, batch_size):
        state_vector = []
        target_f_vector = []
        absolute_errors = [] 

        idx = np.random.randint(len(self.memory), size=batch_size,dtype=int)
        minibatch = np.array(self.memory)[idx]
        
        state, action, reward, next_state = np.concatenate(minibatch[:,0], axis=0 ), minibatch[:,1].astype('int32') ,minibatch[:,2].reshape(batch_size,1), np.concatenate( minibatch[:,3] , axis=0 )
        
        target = reward + self.gamma * np.max(self.target_model.predict(np.reshape(next_state,(batch_size,self.state_size))),axis=1).reshape(batch_size,1)

        # This section incorporates the reward into the prediction and calculates the absolute error between old and new
        target_f = self.model.predict(state)
        
        target_f[np.arange(batch_size),action] = target.reshape(batch_size)
        
        
        #self.model.fit(state_matrix, target_f_matrix, epochs=1, verbose=0)
        self.model.fit(state, target_f, epochs=1, verbose=2,batch_size=batch_size)
        
        self.loss.append(self.model.history.history['loss'])

#         if self.PER_activated:
#             #Update priority
#             self.memory.batch_update(tree_idx, absolute_errors)

        # Copy weights every "copy_weights_frequency" episodes
        #if (episode+1) % self.copy_weights_frequency == 0 and episode != 0:
        #    self.copy_weights()   

    # Copy weights function
    def copy_weights(self):
        self.target_model.set_weights(self.model.get_weights())
        print("Weights succesfully copied to Target model.")  


In [ ]:
class DQN_Environment_Single_Cross():
    def __init__(self,Vissim,npa):
        self.Vissim = Vissim
        self.tsars_dict = defaultdict(defaultdict(int))
        for ID in 
        
        self.action_dict =dict()
    
    def step(self): 
        self.Vissim.Simulation.RunSingleStep()
        
    def # We should script those function later because it is only for the basic intersection here
    
    
    def green_red_to_amber(agent, seconds_per_yellow,Surtrac=False):
        # Fetch the meaning of the Actions from the compatible Actions in the Agent
        previous_action = agent.compatible_actions[agent.action]
        #print("Previous action {}:".format(agent.action+1) + str(previous_action))
        current_action = agent.compatible_actions[agent.newaction]
        #print("Current action {}:".format(agent.newaction+1) + str(current_action))

        # Check transition vector for the whole intersection (1, 0 or -1)
        agent.transition_vector = np.subtract(previous_action, current_action)
        #print("Transition vect:" + str(agent.transition_vector))

        # Cycle through the groups and start the transition
        for index_group, sig_group in enumerate(agent.signal_groups):
            # If the transition vector is > 0, we are changing from GREEN to RED, so set AMBER
            if agent.transition_vector[index_group] == 1:
                sig_group.SetAttValue("SigState", "AMBER")
                #print("Changing Light {} to Red".format(index_group+1))

            # If the transition vector is < 0, we are changing from RED to GREEN, so set to REDAMBER
            elif agent.transition_vector[index_group] == -1:
                #sig_group.SetAttValue("SigState", "REDAMBER")
                #print("Changing Light {} to Green".format(index_group+1))
                pass
            # If the transition vector is zero, the phase stays the same
            elif agent.transition_vector[index_group] == 0:
                pass
            else:
                raise Exception("ERROR: Incongruent new phase and previous phase. Please review the code.")

        # Extend timer after transition is started
        agent.update_counter += seconds_per_yellow	 - 1
        if Surtrac:
            agent.actiontime += - seconds_per_yellow                        
        # Record that a transition is happening
        agent.intermediate_phase = True

    def amber_to_green_red(agent, seconds_per_green,Surtrac=False):
        # Finalize the change
        for index_group, sig_group in enumerate(agent.signal_groups):
            # Use transition vector from previous iteration to finish the change
            if agent.transition_vector[index_group] == 1:
                sig_group.SetAttValue("SigState", "RED")
            elif agent.transition_vector[index_group] == -1:
                sig_group.SetAttValue("SigState", "GREEN")
            elif agent.transition_vector[index_group] == 0:
                pass
            else:
                raise Exception("ERROR: Incongruent new phase and previous phase. Please review the code.")

        # Mark the transition as finished
        agent.intermediate_phase = False	
        # Set timer for next update 
        if Surtrac :
            agent.update_counter += agent.actiontime - 1		
        else :
            agent.update_counter += seconds_per_green - 1
    
    def _calculate_state(Vissim, state_size):
            #Obtain Queue Values (average value over the last period)
            West_Queue  = Vissim.Net.QueueCounters.ItemByKey(1).AttValue('QLen(Current,Last)')
            South_Queue = Vissim.Net.QueueCounters.ItemByKey(2).AttValue('QLen(Current,Last)')
            East_Queue  = Vissim.Net.QueueCounters.ItemByKey(3).AttValue('QLen(Current,Last)')
            North_Queue = Vissim.Net.QueueCounters.ItemByKey(4).AttValue('QLen(Current,Last)')
            state = [West_Queue, South_Queue, East_Queue, North_Queue]
            state = [0. if state is None else state for state in state]
            state = np.reshape(state, [1,state_size])
            return(state)


## Signal Control Unit and Cyclic

In [349]:
# SCU

'''
Calculate state and reward needs updating
    -- multi-junction case this needs updating
'''

# Compatible actions is now a dictionary 
compatible_actions = {0 : [0,1,0,1],\
                      1 : [1,0,1,0]}  


'''
Signal_Control_Unit :
    
    interfaces between a signal controller (at a junction) and actions (provided by an agent)
    
    inputs:
    -- Vissim
    -- Signal_Controller - a Vissim, signal controller
    -- compatible_actions - a dictionary taking IDs to vectors non-conflicting signal groups
    -- green_time = 10   (times are in seconds but converted to simulation steps)
    -- redamber_time = 1
    -- amber_time = 3
    -- red_time = 1
     
    methods :
    -- action_update():
            initiates the change to a new action
            
            inputs:
            -- action_key 
            -- green_time 
    -- update():
            ensures each signal is correct at each simulation step
'''
class Signal_Control_Unit:
    
    def __init__(self,\
                 Vissim,\
                 Signal_Controller,\
                 compatible_actions,\
                 Signal_Groups = None,\
                 green_time = 40,\
                 redamber_time = 1,\
                 amber_time = 3, \
                 red_time = 1\
                ):
        
        # get Vissim, signal controller and its signal groups
        self.Vissim = Vissim
        self.signal_controller = Signal_Controller
        
        if Signal_Groups is None :
            self.signal_groups = self.signal_controller.SGs
        else :
            self.signal_groups = Signal_Groups
            
        # get stae and reward parameters
        self.state = self.calculate_state()
        self.reward = self.calculate_reward()  
       
        self.compatible_actions = compatible_actions
          
        self.time_steps_per_second = self.Vissim.Simulation.AttValue('SimRes')
        
        self.green_time = green_time * self.time_steps_per_second
        self.redamber_time = redamber_time * self.time_steps_per_second
        self.amber_time = amber_time * self.time_steps_per_second
        self.red_time = red_time * self.time_steps_per_second
    
        # implement 1st action to start
        self.action_key = 0   # dict key of current action (we start with 0) 
        self.action_required = False # used to requests an action from agent
        self.update_counter = 0
        self.intermediate_phase = True # tracks when initiating a new action
        self.action_update(self.action_key)    
        

        self.stage = "Green" # tracks the stage particularly when in intermediate phase.
                             # Stages appear in order: "Amber" -> "Red" -> "RedAmber" -> "Green"

            
    '''
    sars :
    returns state, id of action, reward
    '''     
    def sars(self):
        self.state = self.calculate_state()
        self.reward = self.calculate_reward()
        
        return self.state, self.action_key, self.reward

    
    '''
    calculate_state:
    Alvaro's reward function needs to be more general
    '''
    def calculate_state(self,length=None,verbose = False):
        
        Queues = []
        Lanes = []
        for sg in self.signal_groups :
            q = 0 
            for sh in sg.SigHeads:
                if (sh.Lane.AttValue('Link'),sh.Lane.AttValue('Index')) not in Lanes :
                    Lanes.append((sh.Lane.AttValue('Link'),sh.Lane.AttValue('Index')))
                    for veh in sh.Lane.Vehs:
                        q += veh.AttValue('InQueue')
            Queues.append(q)
            # Summarize queue size in each lane
            if verbose :
                print(self.signal_controller.AttValue('No'),sg.AttValue('No'),q)
            
        # now reshape
        if length is not None :
            state = np.reshape(Queues,[1,length])
        else :
            state = np.reshape(Queues,[1,len(Queues)])
        
        return (state)

    
    '''
    calculate_reward:
    Alvaro's reward function needs to be more general
    '''
    def calculate_reward(self):
        state = self.calculate_state()
        reward = -np.sum(state)
        
        return reward
 

    '''
    action_update :
    initiates a new action
        inputs:
        -- id of action
        -- green_time, if specified by agent (in seconds)
    '''    
    def action_update(self,action_key,green_time=None):
        self.intermediate_phase = True # initate intermediate_phase
        self.update_counter = 0 # set update counter zero (will get reset at self.update() )
        self.action_key = action_key
        self.current_action = self.compatible_actions[action_key] 
        self.new_colors = [ 2*val for val in self.current_action] # converts action to 0,1,2 range
        
        if green_time is not None:
            self.green_time = green_time * self.time_steps_per_second
        

    # internal helper function
    # red = 0, amber/redamber = 1 and green = 2
    def _color_convert(self,color):
        if color == "RED" :
            return 0
        elif color == "GREEN" :
            return 2
        else :
            return 1

        
    '''
    _color_changer :
    Internal function
    Changes color of a signal group
        inputs:
        -- signal group
        -- new_color : 2 = green / 0 = red
        -- stage : what stage all lights in the controller are.
    '''          
    def _color_changer(self,signal_group,new_color,stage):
        #Get the current color
        current_color = self._color_convert(signal_group.AttValue("SigState"))
        change = new_color-current_color
        
        # want green but currently red
        if change == -2 and stage == "Green" :
            signal_group.SetAttValue("SigState", "AMBER")
        
        # want red but currently amber
        # if just gone red need on second before green change
        elif change == -1 and stage == "Amber" :
            signal_group.SetAttValue("SigState", "RED")
        
        # want green but currently red 
        elif change == 2 and stage == "Red" :
            signal_group.SetAttValue("SigState", "REDAMBER")
                
        # want green but currently redamber
        elif change == 1 and stage == "RedAmber":
            signal_group.SetAttValue("SigState", "GREEN")
        
        # if both red or green pass (i.e. no change keep green)
        elif change == 0 :
            pass
    

    '''
    _stage_changer :
    Internal function
    
    Track controllers stage (in the stages of Amber->Red->RedAmber-Green) 
    and time for each transtion
    
        inputs:
        -- stage
        
    Nb. stage is a controller method while color is a sg property
    '''
    def _stage_changer(self,stage):
        
        if stage == "Green" :
            time = self.amber_time
            self.stage = "Amber" 
    
        elif stage == "Amber" :
            time = self.red_time
            self.stage = "Red"
        
        elif stage == "Red" :
            time = self.redamber_time
            self.stage = "RedAmber"
                
        # want green but currently redamber
        elif stage == "RedAmber" :
            time = self.green_time
            self.stage = "Green"
        
        return time
                
        
    '''
    update :
    
    returns True if action required (otherwise is None)
    
    implements cycle at each signal group
    and updates the stage of the controllers.
    
    (writen so multiple controllers can be updated in parallel)
    (Computational Overhead should be lower than before)
    
    '''   
    def update(self):
    
        self.update_counter -= 1
        
        # These 'if' clauses mean update computation only happens if needed
        if self.update_counter <= 0 :
            # if update counter just went zero 
            # then ask for an action 
            if self.intermediate_phase is False :
                self.action_required = True 
                return self.action_required
                    
            # if during a change
            # then make the change
            if self.intermediate_phase is True : 
                self.action_reqired = False
                
                # Get light color right for each signal group
                for sg in self.signal_groups :
                    ID = sg.AttValue('No')-1
                    self._color_changer(sg,self.new_colors[ID],self.stage)
                        
                # change the current stage and get time the stage last for
                time = self._stage_changer(self.stage)
                self.update_counter = time
                    
                # if full transition (Amber->Red->RedAmber-Green) to green done  
                if self.stage == "Green" :
                    self.intermediate_phase = False # record current action is implemented
                 
    

In [ ]:
# Basic Simulation Environment
'''
It is assumed that vissim is loaded
'''

class Simulation_Environment():
    def __init__(self,\
                 Loaded_Vissim,\
                 sim_length=3601,\
                 timesteps_per_second=12,\
                 Quick_Mode=True):
        
        self.Vissim = Loaded_Vissim
        self.Quick_Mode(Quick_Mode)
            
        
    def Step(self,action,sim_steps=10):
        
        state = _calculate_state(self)
        reward = _calculate_reward(self)
    
        return  state, reward, done
        
    def _calculate_state(self):
        pass
    
    def _calculate_reward(self):
        pass
    
    def Quick_Mode(self,Quick_Mode):
        self.Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode",Quick_Mode)

In [ ]:
class Env_SPU(Simulation_Environment):
    def __init__(self,\
                 Loaded_Vissim,\
                 sim_length=3601,\
                 timesteps_per_second=1,\
                ):
    
        # Load init from Parent Class
        super(Env_SPU,self).__init__(
                 Loaded_Vissim,\
                 sim_length,\
                 timesteps_per_second,\
                 delete_results,\
                 verbose)
        
            def __init__(self,\
                 Vissim,\
                 Signal_Controller,\
                 compatible_actions,\
                 Signal_Groups = None,\
                 green_time = 40,\
                 redamber_time = 1,\
                 amber_time = 3, \
                 red_time = 1\
                ):
        

In [ ]:
class Cyclic_Control():
    def __init__(self,size):
        self.action = 0
        self.size = size
        
    def choose_action(self,state=None):
        self.action = (self.action + 1) % self.size
        return self.action

### Triple Cross

In [ ]:
# Load Network
model_name  = 'Single_Cross_Triple'
vissim_working_directory = 'C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\Vissim\\'
#vissim_working_directory = 'C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\0_TMF\\MLforFlowOptimisation\\Vissim\\'
compatible_actions = { 0 : [1,1,1,0,0,0,0,0,0,0,0,0],
                       1 : [0,0,0,1,1,1,0,0,0,0,0,0],
                       2 : [0,0,0,0,0,0,1,1,1,0,0,0],
                       3 : [0,0,0,0,0,0,0,0,0,1,1,1],
                       4 : [1,0,0,0,0,0,1,0,0,0,0,0],
                       5 : [0,0,0,1,0,0,0,0,0,1,0,0],
                       6 : [0,1,1,0,0,0,0,1,1,0,0,0],
                       7 : [0,0,0,0,1,1,0,0,0,0,1,1]}

In [ ]:
# MANUAL LOAD (part 2)
input_file = 'C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\Vissim\\Single_Cross_Triple\\Single_Cross_Triple.inpx'
Vissim.LoadNet(input_file)
Env = SE.Q_Sim_Env(Vissim,model_name,vissim_working_directory)

In [ ]:
# Setup
CC = Cyclic_Control(8)

SPU = Signal_Control_Unit(Vissim,\
                          Vissim.Net.SignalControllers[0],\
                          compatible_actions)

In [ ]:
# Main loop
for _ in range(500):  
    SCU.Vissim.Simulation.RunSingleStep()
    action_required = SCU.update()
    if action_required :
        new_action = CC.choose_action()
        SCU.action_update(new_action)

# Balance Network

In [350]:
# Reload Vissim
for _ in range(5):
    try:
        Vissim = com.Dispatch("Vissim.Vissim")
        print('success!')
        break
    except:
        print('fail')

fail
success!


In [351]:
# Load Network
#input_file = 'C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\Vissim\\Balance\\Balance.inpx'
#input_file = 'C:\\Users\\Public\\Documents\\PTV Vision\\PTV Vissim 11\\Examples Training\\Signal Control\\UTC - Workflow PTV Balance PTV Epics\\03 AFTER In PTV Vissim\\PTV Balance PTV Epics Vision Suite Workflow.inpx'
input_file = 'C:\\Users\\nwalton\\OneDrive - The Alan Turing Institute\\Documents\\MLforFlowOptimisation\\Vissim\\NSW\\Balance_NSW\\Balance.inpx'
Vissim.LoadNet(input_file)
#Env = SE.Q_Sim_Env(Vissim,model_name,vissim_working_directory)

In [352]:
# Environment Class
'''
THIS IS A WORK IN PROGRESS MULTIAGENT SIMULATION ENVIRONMENT
'''

class Env_MultiAgent(Simulation_Environment):
    def __init__(self,\
                 Loaded_Vissim,\
                 Controllers_Actions,\
                 Signal_Controllers=None,\
                 sim_length=3601,\
                 timesteps_per_second=1,\
                ):
    
        # Load init from Parent Class
        super(Env_MultiAgent,self).__init__(
                 Loaded_Vissim,\
                 sim_length,\
                 timesteps_per_second)
        

        self.Vissim = Loaded_Vissim


        #self.Quick_Mode(Quick_Mode)
        
        # Signal_Controllers is all controllers or a specified list    
        if Signal_Controllers is not None :
            self.Signal_Controllers = Signal_Controllers
        else : 
            self.Signal_Controllers = Vissim.Net.SignalControllers
        
        self.Controllers_Actions = Controllers_Actions 
        self.SCUs = self._Load_SCUs()
        
    '''
    _Load_SCUs :
        provides a dictionary with at the SCUs
    '''
    def _Load_SCUs(self):
        
        SCUs = dict()
        
        for idx, sc in enumerate(self.Signal_Controllers):
            SCUs[idx] = Signal_Control_Unit(Vissim, sc, self.Controllers_Actions[idx])
        
        return SCUs
    
    '''
    Make a Step
    '''   
    def Step(self,action):
        self.Vissim.Simulation.RunSingleStep()
        
        Sars = dict()
        
        for idx, scu in self.SCUs:
            scu.action_required = SCU.update()
            if action_required :
                Sars[idx] = scu.sars()
        
        if len(actions_required) > 0 :
            return True, Sars
        else:
            return False, None
    
    def sar(self):
        sar_all = dict()
        
        for idx, scu in self.SCUs:
            sar_all[idx] = (scu._calculate_state, scu.action_key, scu._calculate_reward) 
        
        return sar_all
        
    def _calculate_state(self):
        '''
        THIS SHOULD GET QUEUE LENGTH FOR EACH Signal Control Group
        Debug because of multiple signal heads
        '''

        pass
    
#     def _calculate_reward(self):
#         pass
    
#     def Quick_Mode(self,Quick_Mode):
#         self.Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode",Quick_Mode)

NameError: name 'Simulation_Environment' is not defined

In [353]:
# Load Controllers, agents and Environment
Signal_Controllers = Vissim.Net.SignalControllers

MultiAgent = { idx : Cyclic_Control(len(sc.SGs)) for idx, sc in enumerate(Signal_Controllers) }

Env =  Env_MultiAgent(\
                 Vissim,\
                 Controllers_Actions\
                )

NameError: name 'Env_MultiAgent' is not defined

### Debug

In [ ]:
# Load up an SPU

In [109]:
Signal_Controller = Vissim.Net.SignalControllers[0]

In [110]:
Signal_Controller.AttValue('No')

1

In [114]:
len(Signal_Controller.SGs)

12

In [217]:
compatible_actions ={   0 : [ 1, 0, 0, 1, 0, 0,0,0,0,0,0,0],
            1 : [ 0, 1, 0, 0, 1, 0,0,0,0,0,0,0],
            2 : [ 0, 0, 1, 0, 0, 1,0,0,0,0,0,0]
        }

In [140]:
CC = Cyclic_Control(len(compatible_actions))

In [144]:
SCU = Signal_Control_Unit(
                 Vissim,\
                 Signal_Controller,\
                 compatible_actions,\
                 Signal_Groups = None,\
                 green_time = 10,\
                 redamber_time = 1,\
                 amber_time = 3, \
                 red_time = 1\
                )

In [138]:
len(SCU.signal_groups)

12

In [120]:
SCU.action_update(0)

In [145]:
# Main loop
for _ in range(1000):  
    SCU.Vissim.Simulation.RunSingleStep()
    action_required = SCU.update()
    if action_required :
        new_action = CC.choose_action()
        SCU.action_update(new_action)

In [137]:
compatible_actions[SCU.action_key]

[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

#### Debug Multijunction

In [302]:
sc_actions = [None,None]
Signal_Controller = [None, None]
Signal_Groups = [None, None]
SCU = [None, None]

In [303]:
sc_actions[0] ={   0 : [ 1, 0, 0, 1, 0, 0],
            1 : [ 0, 1, 0, 0, 1, 0],
            2 : [ 0, 0, 1, 0, 0, 1]
        }
Signal_Controller[0] = Vissim.Net.SignalControllers[0]

In [304]:
Signal_Groups[0] = []
for i in range(6):
    Signal_Groups[0].append(Signal_Controller[0].SGs[i])

In [305]:
scu1 = Signal_Control_Unit(
                 Vissim,\
                 Signal_Controller[0],\
                 sc_actions[0],\
                 Signal_Groups = Signal_Groups[0],\
                 green_time = 5,\
                 redamber_time = 1,\
                 amber_time = 3, \
                 red_time = 1\
                )
SCU[0] =scu1

In [306]:
sc_actions[1] ={   0 : [ 1, 0, 0, 1, 0, 0],
            1 : [ 0, 1, 0, 0, 1, 0],
            2 : [ 0, 0, 1, 0, 0, 1]
        }
Signal_Controller[1] = Vissim.Net.SignalControllers[1]

In [307]:
Signal_Groups[1] = []
for i in range(6):
    Signal_Groups[1].append(Signal_Controller[1].SGs[i])

In [308]:
scu2 = Signal_Control_Unit(
                 Vissim,\
                 Signal_Controller[1],\
                 sc_actions[1],\
                 Signal_Groups = Signal_Groups[1],\
                 green_time = 5,\
                 redamber_time = 1,\
                 amber_time = 3, \
                 red_time = 1\
                )
SCU[1] =scu2

In [311]:
CC = [ Cyclic_Control(3),  Cyclic_Control(3)] 

In [309]:
import time
#time.sleep(2)

In [312]:
# Main loop
for _ in range(1000):  
    Vissim.Simulation.RunSingleStep()
   # time.sleep(0.3)
    for i,scu in enumerate(SCU) :
        action_required = scu.update()
        if action_required :
            new_action = CC[i].choose_action()
            scu.action_update(new_action)

### Debug All Junctions cyclic

In [355]:
# all controller actions
Controllers_Actions =\
{\
    # Controller Number 2 
    0 : {   0 : [ 1, 0, 0, 1, 0, 0,],
            1 : [ 0, 1, 0, 0, 1, 0,],
            2 : [ 0, 0, 1, 0, 0, 1,]
        },
    # Controller Number 3
    1 : {   0 : [ 1, 0, 0, 1, 0, 0,],
            1 : [ 0, 1, 0, 0, 1, 0,],
            2 : [ 0, 0, 1, 0, 0, 1,]
        },
    # Controller Number 4
    2 : {
            0 : [1,0,0,0,1,0,0,0],
            1 : [0,1,0,0,0,1,0,0],
            2 : [0,0,1,0,0,0,1,0],
            3 : [0,0,0,1,0,0,0,1]
        },
    # Controller Number 5
    3 : {
            0 : [1,0,0,1,0,0],
            1 : [0,1,0,0,1,0],
            2 : [0,0,1,0,0,1],
        },
    # Controller Number 6 
    4 : {
            0 : [1,1,0,0],
            1 : [0,1,1,0],
            2 : [1,0,0,1]
        },
    # Controller Number 8
    5 : {
            0 : [1,0,1],
            1 : [0,1,0]
        },
    # Controller Number 9
    6 : {
            0 : [1,0,1,0],
            1 : [0,1,0,1]
        },
    # Contoller Number 10
    7 : {
            0 : [1,0,0,0,0],
            1 : [0,1,0,1,0],
            2 : [0,0,1,0,1]
        },
    # Controller Number 12
    8 : {
            0 : [1,0,1],
            1 : [0,1,0],
        },
    # Controller Number 13
    9 : {
            0 : [1,0,0,0],
            1 : [0,1,0,1],
            1 : [0,0,1,0],
        },
    # Controller 15
     10 : {   
            0 : [1,0,1,0],\
            1 : [0,1,0,1]\
        },
    # Controller 16
    11 : {   
            0 : [1,0,1,0],\
            1 : [0,1,0,1]\
        },
    # Controller 17
    12 : {
            0 : [1,0,0,0,0],
            1 : [0,1,0,0,1],
            2 : [0,0,1,0,0],
            3 : [0,0,0,1,1],
            4 : [0,1,0,0,1]
        },
    # Controller 33 
    13 : {
            0 : [1,0,0,1],
            1 : [1,0,1,0],
            2 : [0,1,0,0]
        }
}

In [356]:
number_of_scs = len(Controllers_Actions)

Signal_Controller = []
for idx in range(number_of_scs):
    Signal_Controller.append(Vissim.Net.SignalControllers[idx])

Signal_Groups = [ [] for _ in range(number_of_scs)]

for i, SGs in enumerate(Signal_Groups) :  
    for j in range(len(Controllers_Actions[i][0])):
        SGs.append(Signal_Controller[i].SGs[j])
        #print(Signal_Controller[i].SGs[j].AttValue('No'))

SCU =  []

for i in range(number_of_scs):
    SCU.append(\
              Signal_Control_Unit(
                 Vissim,\
                 Signal_Controller[i],\
                 Controllers_Actions[i],\
                 Signal_Groups = Signal_Groups[i],\
                 green_time = 5,\
                 redamber_time = 1,\
                 amber_time = 3, \
                 red_time = 1\
                )\
              )

CC =[]
for i, acts in enumerate(Controllers_Actions):
    cycle_size = len(Controllers_Actions[i])
    CC.append(Cyclic_Control(cycle_size))

In [362]:
# Main loop
for _ in range(4000):  
    Vissim.Simulation.RunSingleStep()
   # time.sleep(0.3)
    for i, scu in enumerate(SCU) :
        action_required = scu.update()
        if action_required :
            # state = scu.calculate_state()
            new_action = CC[i].choose_action()
            scu.action_update(new_action)

com_error: (-2147352567, 'Exception occurred.', (0, 'VISSIM.Vissim.1100', 'AttValue failed: Object 1 - 1: SG1L: Attribute Signal state is no subject to changes.', None, 0, -2147352567), None)

In [331]:
SCU[0].action_key

0

In [332]:
SCU[0].compatible_actions

{0: [1, 0, 0, 1, 0, 0], 1: [0, 1, 0, 0, 1, 0], 2: [0, 0, 1, 0, 0, 1]}

In [333]:
SCU[0].current_action

[1, 0, 0, 1, 0, 0]

In [335]:
SCU[0].new_colors

[2, 0, 0, 2, 0, 0]

In [334]:
for sg in SCU[0].signal_groups:
    print(sg.AttValue('No'))

1
2
3
4
5
6


In [326]:
Signal_Groups[0]

[<COMObject <unknown>>, <COMObject <unknown>>, <COMObject <unknown>>]

## More debugging

In [ ]:
'''
Idea for main control loop
'''

action_required, scu_sars = Env.Step()

if action_required :
    for idx, sar in enumerate(scu_sars):
        state, action, reward = sar
        action = Agent.choose_action(state)
        Env.SCUs[idx]. action
    


In [ ]:
Vissim.Net.QueueCounters.ItemByKey(4).AttValue('QLen(Current,Last)')

In [ ]:
len(Vissim.Net.QueueCounters)

In [ ]:
for i in range(len(Vissim.Net.Links)):
    print(len(Vissim.Net.Links[i].QueueCounters))

In [ ]:
Vissim.Simulation.RunSingleStep()

In [ ]:
class Env_SPU(Simulation_Environment):
    def __init__(self,\
                 Loaded_Vissim,\
                 sim_length=3601,\
                 timesteps_per_second=1,\
                ):
    
        # Load init from Parent Class
        super(Env_SPU,self).__init__(
                 Loaded_Vissim,\
                 sim_length,\
                 timesteps_per_second,\
                 delete_results,\
                 verbose)
        
            def __init__(self,\
                 Vissim,\
                 Signal_Controller,\
                 compatible_actions,\
                 Signal_Groups = None,\
                 green_time = 40,\
                 redamber_time = 1,\
                 amber_time = 3, \
                 red_time = 1\
                ):
        

In [ ]:
Vissim.Net.SignalControllers[0].SGs[4].SigHeads[0].Lane.AttValue('Link')

In [ ]:
'''
YOU ARE HERE
'''
for veh in Vissim.Net.SignalControllers[0].SGs[4].SigHeads[0].Lane.Vehs:
    print(veh.AttValue('InQueue'))

In [ ]:
'''
THIS SHOULD GET QUEUE LENGTH FOR EACH Signal Control Group
Debug because of multiple signal heads
'''
sc = Vissim.Net.SignalControllers[7]
Queues = []
Lanes = []
for sg in sc.SGs :
    q = 0 
    for sh in sg.SigHeads:
        if (sh.Lane.AttValue('Link'),sh.Lane.AttValue('Index')) not in Lanes :
            Lanes.append((sh.Lane.AttValue('Link'),sh.Lane.AttValue('Index')))
            for veh in sh.Lane.Vehs:
                q += veh.AttValue('InQueue')
    Queues.append(q)
    print(sc.AttValue('No'),sg.AttValue('No'),q)

In [ ]:
Queues

# Debug

In [ ]:
action_required, scu_sars = Env.Step()

## Actions for Balance Network

In [148]:
Controllers_Actions =\
{\
    # Controller Number 2 
    0 : {   0 : [ 1, 0, 0, 1, 0, 0,],
            1 : [ 0, 1, 0, 0, 1, 0,],
            2 : [ 0, 0, 1, 0, 0, 1,]
        },
    # Controller Number 3
    1 : {   0 : [ 1, 0, 0, 1, 0, 0,],
            1 : [ 0, 1, 0, 0, 1, 0,],
            2 : [ 0, 0, 1, 0, 0, 1,]
        },
    # Controller Number 4
    2 : {
            0 : [1,0,0,0,1,0,0,0],
            1 : [0,1,0,0,0,1,0,0],
            2 : [0,0,1,0,0,0,1,0],
            3 : [0,0,0,1,0,0,0,1]
        },
    # Controller Number 5
    3 : {
            0 : [1,0,0,1,0,0],
            1 : [0,1,0,0,1,0],
            2 : [0,0,1,0,0,1],
        },
    # Controller Number 6 
    4 : {
            0 : [1,1,0,0],
            1 : [0,1,1,0],
            2 : [0,0,0,1]
        },
    # Controller Number 8
    5 : {
            0 : [1,0,1],
            1 : [0,1,0]
        },
    # Controller Number 9
    6 : {
            0 : [1,0,1,0],
            1 : [0,1,0,1]
        },
    # Contoller Number 10
    7 : {
            0 : [1,0,0,0,0],
            1 : [0,1,0,1,0],
            2 : [0,0,1,0,1]
        },
    # Controller Number 12
    8 : {
            0 : [1,0,1],
            1 : [0,1,0],
        },
    # Controller Number 13
    9 : {
            0 : [1,0,1],
            1 : [0,1,0],
        },
    # Controller 15
     10 : {   
            0 : [1,0,1,0],\
            1 : [0,1,0,1]\
        },
    # Controller 16
    11 : {   
            0 : [1,0,1,0],\
            1 : [0,1,0,1]\
        },
    # Controller 17
    12 : {
            0 : [1,0,0,0,0],
            1 : [0,1,0,0,1],
            2 : [0,0,1,0,0],
            3 : [0,0,0,1,1],
            4 : [0,1,0,0,1]
        },
    # Controller 33 
    13 : {
            0 : [1,0,0,1],
            1 : [1,0,1,0],
            2 : [0,1,0,0]
        }
}

In [ ]:
len(dict())

In [ ]:
Actions_Controller_2 \
= \
{   0 : [ 1, 0, 0, 1, 0, 0,],\
    1 : [ 0, 1, 0, 0, 1, 0,],\
    2 : [ 0, 1, 0, 0, 1, 0,]\
}

In [ ]:
Actions_Controller_3 = Actions_Controller_2

In [ ]:
Actions_Controller_4 \
=\
{
    0 : [1,0,0,0,1,0,0,0],
    1 : [0,1,0,0,0,1,0,0],
    2 : [0,0,1,0,0,0,1,0],
    3 : [0,0,0,1,0,0,0,1]
}

In [ ]:
Actions_Controller_5 \
=\
{
    0 : [1,0,0,1,0,0],
    1 : [0,1,0,0,1,0],
    2 : [0,0,1,0,0,1],
}

In [ ]:
Actions_Controller_6 \
=\
{
    0 : [1,1,0,0],
    1 : [0,1,1,0],
    2 : [0,0,0,1]
}

In [ ]:
Actions_Controller_8 \
=\
{
    0 : [1,0,1],
    1 : [0,1,0]
}

In [ ]:
for sg in sc_0.SGs :
    print( sg.AttValue('SigState'))

In [ ]:
for idx, sg in enumerate(SG) :
    if idx == 0 or idx == 3 :
        sg.SetAttValue('SigState','GREEN')
    else :
        sg.SetAttValue('SigState','RED')

In [ ]:
len(sc_0.SGs)

In [ ]:
SPU.Vissim.Simulation.RunSingleStep()

In [ ]:
SG = [ sg for idx, sg in enumerate(sc_0.SGs) if idx < 6 ]

In [ ]:
len(SG)

In [ ]:
for sc in Vissim.Net.SignalControllers :
    for sg in sc.SGs :
        print(sc.AttValue('No'),sg.AttValue('No'))

In [ ]:
Actions_Controller_9 \
=\
{
    0 : [1,0,1,0],
    1 : [0,1,0,1]
}

Actions_Controller_10 \
=\
{
    0 : [1,0,0,0,0],
    1 : [0,1,0,1,0],
    2 : [0,0,1,0,1]
}

Actions_Controller_12 = \
{
    0 : [1,0,1],
    1 : [0,1,0],
}

In [ ]:
Actions_Controller_13 = Actions_Controller_15

In [ ]:
Actions_Controller_15 \
= \
{   0 : [1,0,1,0],\
    1 : [0,1,0,1]\
}

In [ ]:
Actions_Controller_16 = Actions_Controller_15

In [ ]:
Actions_Controller_17 \
=\
{
    0 : [1,0,0,0,0],
    1 : [0,1,0,0,1],
    2 : [0,0,1,0,0],
    3 : [0,0,0,1,1],
    4 : [0,1,0,0,1]
}

In [ ]:
Actions_Controller_33 \
=\
{
    0 : [1,0,0,1],
    1 : [1,0,1,0],
    2 : [0,1,0,0]
}

In [ ]:
Controller_Actions[13][0][0]

In [ ]:
Actions_Controller_9 \
=\
{
    0 : [1,0,1,0],
    1 : [0,1,0,1]
}

In [ ]:
CC = Cyclic_Control(8)

In [ ]:
CC.choose_action()

In [ ]:
for sg in Vissim.Net.SignalControllers[0].SGs :
    for sh in sg.SigHeads :
        print(sg.AttValue('No'), sh.AttValue('Lane'))

In [ ]:
SCU = Signal_Control_Unit(Env.Vissim,\
                          Env.Vissim.Net.SignalControllers[0],\
                          compatible_actions)

In [ ]:
for _ in range(500):  
    SCU.Vissim.Simulation.RunSingleStep()
    action_required = SCU.update()
    if action_required :
        new_action = CC.choose_action()
        SCU.action_update(new_action)

In [ ]:
# Run a Single Episode for a set simulation length
def run_simulation_episode(Agents, Vissim, state_type, reward_type,\
 state_size, simulation_length, timesteps_per_second, seconds_per_green,\
  seconds_per_yellow, demand_list, demand_change_timesteps, mode, PER_activated, Surtrac = False, AC = False):
	
	for time_t in range(simulation_length):

		# Change demand every 450 seconds.
		if time_t % demand_change_timesteps == 0:
			change_demand(Vissim, demand_list, demand_change_timesteps, time_t)
		
		# Pass the control over to COM
		if time_t == 1:
			for agent in Agents:
				for group in agent.signal_groups:
					group.SetAttValue('ContrByCOM',1)

		# Cycle through all agents and update them
		Agents_update(Agents, Vissim, state_type,reward_type, state_size, seconds_per_green, seconds_per_yellow, mode, time_t, Surtrac = Surtrac , AC = AC )
		
		# Advance the game to the next second (proportionally to the simulator resolution).
		for _ in range(0, timesteps_per_second):
			Vissim.Simulation.RunSingleStep()

		if mode == "test":
			for agent in Agents:
				agent.queues_over_time.append(get_queue_lengths(Vissim, agent))
				agent.accumulated_delay.append(agent.accumulated_delay[-1]+get_delay_timestep(Vissim))

	# Reconfigure agents for the start of the next episode
	for agent in Agents:
		agent.update_counter = 1
		agent.intermediate_phase = False
		agent.action = 0
	# Stop the simulation    
	Vissim.Simulation.Stop()


In [ ]:
def Agents_update(Agents, Vissim, state_type, reward_type, state_size, seconds_per_green, seconds_per_yellow, mode, time_t, Surtrac = False, AC = False):
	
	for index, agent in enumerate(Agents):
		# Check if agent needs to update
		if agent.update_counter > 0:
			# If it doesn't, substract 1 timestep from the counter and skip the rest
			agent.update_counter -= 1
			continue

		# Update the agent
		elif agent.update_counter == 0:
			if mode	 == "debug":
				print("Update at time {}".format(time_t))
			# Make sure the agent is not in the middle of a transition
			if agent.intermediate_phase == False:
				# Compute the current State and store it in the agent
				agent.newstate = calculate_state(Vissim, state_type, state_size)
				
				# Reward generated by last Action and store it in the agent
				agent.reward   = calculate_reward(Vissim, reward_type)
				
				if not Surtrac :
				# Commit previous State, previous Action, Reward generated and current State to memory
					agent.remember(agent.state, agent.action, agent.reward, agent.newstate)
					
				agent.episode_reward.append(agent.reward)		
				#print(agent.newstate)

				# Compute the new Action and store it in the agent
				agent.newaction = agent.choose_action(agent.newstate)
				
				# In Demonstration Mode, show the Reward of the last cycle
				if mode == "demo":
					print('Agent Reward in this cycle is : {}'.format(round(agent.reward,2)))

				# If the same Action is chosen
				if agent.newaction == agent.action:
					# Extend Timer (do nothing)
					if Surtrac:
						agent.update_counter += agent.actiontime - 1
					else:
						agent.update_counter += seconds_per_green - 1
					#print("Extended Phase")

				# If a different Action is chosen
				elif agent.newaction != agent.action:
					# Transition from green to amber and from red to redamber
					green_red_to_amber(agent, seconds_per_yellow,Surtrac)
					#print("Finished Transition")

			# If the agent is in the middle of a transition
			elif agent.intermediate_phase == True:
				# Transition from amber to red and from redamber to green
				amber_to_green_red(agent, seconds_per_green, Surtrac)

			# Update internal State
			agent.state  = agent.newstate
			# Update internal Action
			agent.action = agent.newaction  
			
			# Training during the episode
			if mode == 'training' and AC :
				agent.trainstep += 1
				if len(agent.memory) == agent.n_step_size and agent.trainstep >= 1:
					agent.learn()
					agent.trainstep = 0				
		# Error protection against negative update counters
		else:
			raise Exception("ERROR: Update Counter for agent {} is negative. Please investigate.".format(index))